<a href="https://colab.research.google.com/github/mrdbourke/pytorch-deep-learning/blob/main/extras/exercises/05_pytorch_going_modular_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:**

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [1]:
%%writefile get_data.py
import requests
import zipfile
from pathlib import Path
import os
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

if image_path.is_dir():
  print(f"{image_path} directory exists.")
else:
  print(f"Did not find {image_path} directory, creating...")
  image_path.mkdir(parents=True, exist_ok=True)

with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print("Downloading pizza, steak, sushi data...")
  f.write(request.content)

with zipfile.ZipFile(data_path/"pizza_steak_sushi.zip", "r") as zip_ref:
  print(f"Unzipping pizza, steak, sushi data to {image_path}")
  zip_ref.extractall(image_path)
os.remove(data_path / "pizza_steak_sushi.zip")

Writing get_data.py


In [2]:
# Example running of get_data.py
!python get_data.py

Did not find data/pizza_steak_sushi directory, creating...
Unzipping pizza, steak, sushi data to data/pizza_steak_sushi


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular).

In [ ]:
going_modular/
├── going_modular/
│   ├── data_setup.py
│   ├── engine.py
│   ├── model_builder.py
│   ├── train.py
│   └── utils.py
├── models/
│   ├── 05_going_modular_cell_mode_tinyvgg_model.pth
│   └── 05_going_modular_script_mode_tinyvgg_model.pth
└── data/
    └── pizza_steak_sushi/
        ├── train/
        │   ├── pizza/
        │   │   ├── image01.jpeg
        │   │   └── ...
        │   ├── steak/
        │   └── sushi/
        └── test/
            ├── pizza/
            ├── steak/
            └── sushi/

In [3]:
%%writefile data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for
image classification data.
"""
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
  """Creates training and testing DataLoaders.
  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.
  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.
  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                             test_dir=path/to/test_dir,
                             transform=some_transform,
                             batch_size=32,
                             num_workers=4)
  """
  # Use ImageFolder to create dataset(s)
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  # Get class names
  class_names = train_data.classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names

Writing data_setup.py


In [5]:
%%writefile engine.py
"""
Contains functions for training and testing a PyTorch model.
"""
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
   # Put the model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader and data batches
  for batch, (X, y) in enumerate(dataloader):
    # Send data to target device
    X, y = X.to(device), y.to(device)
    # 1. Forward pass
    y_pred = model(X)
    # 2. Calculate and accumulate loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    # Calculate and accumualte accuracy metric across all batches
    y_pred_res = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_res == y).sum().item()/len(y_pred)
  # Adjust metrics to get average loss and average accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  # Put model in eval mode
  model.eval()

  # Setup the test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
    # Loop through DataLoader batches
    for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      test_pred_logits = model(X)
      # print(test_pred_logits)

      # 2. Calculuate and accumulate loss
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()

      # Calculate and accumulate accuracy
      test_pred_res = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_res == y).sum().item()/len(test_pred_res))

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results

Writing engine.py


In [4]:
%%writefile model_builder.py
#Реализуем прошлую модель
import torch
from torch import nn
class FIRST_MY_MODEL(torch.nn.Module):
  """The model from the last lesson
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
    super().__init__()
    self.first_block = nn.Sequential(
        #Сначала сверточный слой для двумерных данных
        nn.Conv2d(in_channels= input_shape,
                  out_channels= hidden_units,
                  kernel_size= 3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels= hidden_units,
                  out_channels= hidden_units,
                  kernel_size= 3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2))
    self.second_block = nn.Sequential(
                  nn.Conv2d(in_channels= hidden_units,
                  out_channels= hidden_units,
                  kernel_size= 3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels= hidden_units,
                  out_channels= hidden_units,
                  kernel_size= 3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2))
    self.result_classification = nn.Sequential(
      nn.Flatten(),
      nn.Linear(in_features=hidden_units*int((64 /2 /2)) * int((64 /2 /2)),
                out_features=output_shape)
    )
  def forward(self, x):
    x = self.first_block(x)
    x = self.second_block(x)
    x = self.result_classification(x)
    return x

Writing model_builder.py


In [6]:
%%writefile utils.py
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)
    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing utils.py


In [7]:
%%writefile train.py

import os
import argparse
import torch
from torchvision import transforms
import data_setup, engine, model_builder, utils

def get_args():
    parser = argparse.ArgumentParser(description="Training hyperparameters.")
    parser.add_argument("--num_epochs", default=10, type=int, help="Number of epochs to train for")
    parser.add_argument("--batch_size", default=32, type=int, help="Number of samples per batch")
    parser.add_argument("--hidden_units", default=10, type=int, help="Number of hidden units in hidden layers")
    parser.add_argument("--learning_rate", default=0.001, type=float, help="Learning rate for the model")
    parser.add_argument("--train_dir", default="data/pizza_steak_sushi/train", type=str, help="Training data directory")
    parser.add_argument("--test_dir", default="data/pizza_steak_sushi/test", type=str, help="Testing data directory")
    return parser.parse_args()

def main():
    args = get_args()

    NUM_EPOCHS = args.num_epochs
    BATCH_SIZE = args.batch_size
    HIDDEN_UNITS = args.hidden_units
    LEARNING_RATE = args.learning_rate

    print(f"[INFO] Training a model for {NUM_EPOCHS} epochs with batch size {BATCH_SIZE}, using {HIDDEN_UNITS} hidden units and a learning rate of {LEARNING_RATE}")
    print(f"[INFO] Training data directory: {args.train_dir}")
    print(f"[INFO] Testing data directory: {args.test_dir}")

    device = "cuda" if torch.cuda.is_available() else "cpu"

    data_transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor()
    ])

    train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
        train_dir=args.train_dir,
        test_dir=args.test_dir,
        transform=data_transform,
        batch_size=BATCH_SIZE
    )

    model = model_builder.FIRST_MY_MODEL(
        input_shape=3,
        hidden_units=HIDDEN_UNITS,
        output_shape=len(class_names)
    ).to(device)

    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    engine.train(
        model=model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        epochs=NUM_EPOCHS,
        device=device
    )

    utils.save_model(
        model=model,
        target_dir="models",
        model_name="05_going_modular_script_mode_tinyvgg_model.pth"
    )

if __name__ == "__main__":
    main()


Writing train.py


In [8]:
# Example running of train.py
!python train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

[INFO] Training a model for 5 epochs with batch size 128, using 128 hidden units and a learning rate of 0.0003
[INFO] Training data directory: data/pizza_steak_sushi/train
[INFO] Testing data directory: data/pizza_steak_sushi/test
  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.0999 | train_acc: 0.3329 | test_loss: 1.1024 | test_acc: 0.3467
 20% 1/5 [00:37<02:30, 37.63s/it]Epoch: 2 | train_loss: 1.0904 | train_acc: 0.4284 | test_loss: 1.0879 | test_acc: 0.4000
 40% 2/5 [01:15<01:52, 37.62s/it]Epoch: 3 | train_loss: 1.0669 | train_acc: 0.4846 | test_loss: 1.0690 | test_acc: 0.4000
 60% 3/5 [01:51<01:14, 37.21s/it]Epoch: 4 | train_loss: 1.0223 | train_acc: 0.5062 | test_loss: 1.0201 | test_acc: 0.5200
 80% 4/5 [02:28<00:36, 36.75s/it]Epoch: 5 | train_loss: 0.9363 | train_acc: 0.6193 | test_loss: 1.0203 | test_acc: 0.5067
100% 5/5 [03:08<00:00, 37.62s/it]
[INFO] Saving model to: models/05_going_modular_script_mode_tinyvgg_model.pth


## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function).
* You may also have to write code to load in a trained model.

In [9]:
%%writefile predict.py

import torch
import torchvision
import argparse
import model_builder

def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--image", help="Target image filepath to predict on")
    parser.add_argument("--model_path", default="models/05_going_modular_script_mode_tinyvgg_model.pth", type=str, help="Target model filepath for prediction")
    return parser.parse_args()

def load_model(filepath, device):
    model = model_builder.FIRST_MY_MODEL(input_shape=3, hidden_units=128, output_shape=3).to(device)
    print(f"[INFO] Loading model from: {filepath}")
    model.load_state_dict(torch.load(filepath))
    return model

def predict_on_image(image_path, model, device):
    class_names = ["pizza", "steak", "sushi"]
    image = torchvision.io.read_image(str(image_path)).type(torch.float32)
    image = image / 255.0
    transform = torchvision.transforms.Resize(size=(64, 64))
    image = transform(image)

    model.eval()
    with torch.inference_mode():
        image = image.to(device)
        pred_logits = model(image.unsqueeze(dim=0))
        pred_prob = torch.softmax(pred_logits, dim=1)
        pred_label = torch.argmax(pred_prob, dim=1)
        pred_label_class = class_names[pred_label]

    print(f"[INFO] Pred class: {pred_label_class}, Pred prob: {pred_prob.max():.3f}")

def main():
    args = get_args()
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"[INFO] Predicting on {args.image}")
    model = load_model(filepath=args.model_path, device=device)
    predict_on_image(image_path=args.image, model=model, device=device)

if __name__ == "__main__":
    main()


Writing predict.py


In [10]:
# Example running of predict.py
!python predict.py --image data/pizza_steak_sushi/test/sushi/175783.jpg

[INFO] Predicting on data/pizza_steak_sushi/test/sushi/175783.jpg
[INFO] Loading model from: models/05_going_modular_script_mode_tinyvgg_model.pth
[INFO] Pred class: sushi, Pred prob: 0.480
